<h3> Run sheet for end user notebooks code refactoring..

In [ ]:
import os, re, fnmatch
import pandas as pd
from helper_functions import *

In [ ]:

#update env parameter and run for various environments
env = "dev_"

In [ ]:
# os.mkdir(f"./user_notebooks/{env.strip('_')}/zips/")
# os.mkdir(f"./user_notebooks/{env.strip('_')}/notebooks/")

In [ ]:
#prep folders for pulling end user notebooks
try:    
    os.mkdir(f"./user_notebooks/{env.strip('_')}/")
except Exception as e:
    print (e)

Update the parameter to correct env in "pull_enduser_files.py" and execute the script to extract all the files from end user workspace folders.

In [ ]:
def get_raw_strings_to_replace(full_path):
    """get list of strings in codebase that contains 'raw.' (to be later replaced with newer namepsace)"""
    global strings_to_replace
    pattern = re.compile(r"\braw\.\b")
    try:
        for filename in os.listdir(full_path):
            if filename.endswith(".py") or filename.endswith(".sql"):
                with open(os.path.join(full_path, filename), "r") as f:
                    for line in f:
                        for word in line.split():
                            if pattern.search(word): 
                                if (len(word.split("."))) == 2:
                                    strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                                else:
                                    print (f"This was formatted poorly and is hence not included in the list:\n {word}\n")
            elif os.path.isdir(os.path.join(full_path, filename)) and 'Trash' not in filename:
                get_raw_strings_to_replace(os.path.join(full_path, filename))
            else:
                pass
    except Exception as e:
        pass

In [ ]:
def get_cleansed_strings_to_replace(full_path):
    """get list of strings in codebase that contains 'cleansed.' (to be later replaced with newer namepsace)"""
    global strings_to_replace
    pattern = re.compile(r"\bcleansed\.\b")
    try:
        for filename in os.listdir(full_path):
            if filename.endswith(".py") or filename.endswith(".sql"):
                with open(os.path.join(full_path, filename), "r") as f:
                    for line in f:
                        for word in line.split():
                            if pattern.search(word): 
                                print (word)
                                print (full_path, filename)
                                if (len(word.split("."))) == 2:
                                    strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                                else:
                                    print (f"This was formatted poorly and is hence not included in the list:\n {word}\n")
            elif os.path.isdir(os.path.join(full_path, filename)) and 'Trash' not in filename:
                get_cleansed_strings_to_replace(os.path.join(full_path, filename))
            else:
                pass
    except Exception as e:
        pass

In [ ]:
def get_semantic_strings_to_replace(full_path):
    """get list of strings in codebase that contains 'semantic.' (to be later replaced with newer namepsace)"""
    global strings_to_replace
    pattern = re.compile(r"\bsemantic\.\b")
    try:
        for filename in os.listdir(full_path):
            if filename.endswith(".py") or filename.endswith(".sql"):
                with open(os.path.join(full_path, filename), "r") as f:
                    for line in f:
                        for word in line.split():
                            if pattern.search(word): 
                                if (len(word.split("."))) == 2:
                                    strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                                else:
                                    print (f"This was formatted poorly and is hence not included in the list:\n {word}\n")
            elif os.path.isdir(os.path.join(full_path, filename)) and 'Trash' not in filename:
                get_semantic_strings_to_replace(os.path.join(full_path, filename))
            else:
                pass
    except Exception as e:
        pass

In [ ]:
def get_curated_strings_to_replace(full_path):
    """get list of strings in codebase that contains 'curated.' (to be later replaced with newer namepsace)"""
    global strings_to_replace
    pattern = re.compile(r"\bcurated\.\b")
    try:
        for filename in os.listdir(full_path):
            if filename.endswith(".py") or filename.endswith(".sql"):
                with open(os.path.join(full_path, filename), "r") as f:
                    for line in f:
                        for word in line.split():
                            if pattern.search(word): 
                                if (len(word.split("."))) == 2:
                                    strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                                else:
                                    print (f"This was formatted poorly and is hence not included in the list:\n {word}\n")
            elif os.path.isdir(os.path.join(full_path, filename)) and 'Trash' not in filename:
                get_curated_strings_to_replace(os.path.join(full_path, filename))
            else:
                pass
    except Exception as e:
        pass

In [ ]:
strings_to_replace = []

In [ ]:
#iterate through the notebooks and collect list of strings to modify
for item in os.listdir(os.path.join(os.getcwd(), 'user_notebooks', 'dev', 'notebooks')):
    if '.zip' not in item:
        path = os.path.join(os.getcwd(), 'user_notebooks', 'dev', 'notebooks', item)
        # print (path)
        get_cleansed_strings_to_replace(path)
        get_curated_strings_to_replace(path)
        get_raw_strings_to_replace(path)
        get_semantic_strings_to_replace(path)

In [ ]:
strings_to_replace = list(set(strings_to_replace))

In [ ]:
#create pandas dataframe out of the strings_to_replace
df = pd.DataFrame({'current_namespace': strings_to_replace})


In [ ]:
def trim_namespace(given_namespace):
     return given_namespace.replace('__', '').replace('"', '').replace("'", '').replace("--", '').replace(",", '').replace(";", '')

In [ ]:
def update_namespace(given_namespace):
    """provide updated UC namespace for a given hive metastore namespace. use provided excel sheet to override the namespace if available."""
    try:
        db = given_namespace.split('.')[0]
        table = given_namespace.split('.')[1]
        print (db, table)
        namespace_elements = get_target_namespace(env, db, table, excel_path='./uc_scope.xlsx')
        return f"{namespace_elements['catalog_name']}.{namespace_elements['database_name']}.{namespace_elements['table_name']}"
    except Exception as e:
        print (e)
        return None

In [ ]:
#update_namespace('cleansed.sysx_tabley')
# will return something like dev_cleansed.sysx.tabley

In [ ]:
df['trimmed_namespace'] = df['current_namespace'].apply(lambda x: trim_namespace(x))
df['future_namespace'] = df['trimmed_namespace'].apply(lambda x: update_namespace(x))

In [ ]:
for item in df.values:
    if item[2] != None:
        print (f"Replacing item '{item[1]}' with '{item[2]}'")
        find_replace(f'./user_notebooks/dev/notebooks/', '', item[1], item[2])
    # item0 -> current_namespace
    # item1 -> trimmed_namespace
    # item2 -> future_namespace

Once happy with the changes in end user notebooks, push them back into the respective workspace folders by executing push_enduser_files.py. This will push all the updated files back into the workspace user folder (as a zip file). The zip file when received at Databricks workspace, the contents are unzipped. This will also push the original files into a "backup" folder.